# Side Channel Analysis Metric API
There exists very few comprehensive open-source libraries for side-channel analysis. Libraries that do exist are difficult to use and lack the proper documentation. This Jupyter Notebook outlines some of the most useful metrics when conducting side channel analysis in an easy to understand medium. These metrics do not perform an attack, rather they are used to assess gain insight into the cryptographic system. Many of the metrics in this API require data to be pre-formatted prior to use. However, each metric explains what programmer needs to provide. 

In [ ]:
import numpy as np
from scipy import stats
import math

## Signal-to-Noise Ratio 
The signal-to-noise ratio of a signal is defined as the ratio of a signal's data component to the signal's noise component. For side-channel analysis, the SNR of a power trace relates to the ability for an attacker to obtain information from a power trace during an attack. The effectiveness of side channel attack increases for larger SNR values since the signal leakage is more prominent relative to the noise of the signal. Typically recorded power traces need to be partitioned into different sets called labels. 

$$
SNR = \frac{VAR(L_d)}{VAR(L_n)} = \frac{\sum_{v=0}^{V} (\hat{\mu_v}^2 - \hat{\mu})^2}{\hat{\sigma}^2}
$$
The resulting array is the value of the SNR at a given discrete time sample. Windows of the resulting trace where the magnitude of the SNR is high may also indicate an area of interest since it implies that there exists a significant amount of leakage at that sample.

In [8]:
def signal_to_noise_ratio(labels):
    # statistical mean and variances of each set
    set_means = []
    signal_traces = []
    l_n = []

    for trace_set in labels.values():
        set_means.append(np.mean(trace_set, axis=0))  # take the mean along the column #good
        for trace in trace_set:
            l_n.append(trace - set_means[-1])
        signal_traces.append(set_means[-1])

    l_n = np.var(l_n, axis=0)
    l_d = np.var(signal_traces, axis=0)

    snr = np.divide(l_d, l_n)

    return snr

## Score and Rank
The score and rank metric is a helpful metric to use both during an attack and in the analysis of a system. This metric relies primarily on two steps, first, the full-length cryptographic key is split into multiple segments, called partitions. Typically, these partitions are the size of a byte, but can be either larger or shorter depending on the particular encryption algorithm and user implementation. This means that for partitions that are the size of a byte, there are 256 key possibilities. Next, a scoring function needs to be specified. This function is arbitrary but needs to return a numerical scores such that the higher the score, the more likely a given input key, k, actually produced the traces. Using the scoring function, for each partition, each possible key is ranked from the highest score to the lowest score. The idea of ranking and scoring the key guesses is that as the number of traces increases, the rank will converge to the point that the actual key will remain in the 1st rank, or very close to it for all key partitions. 

In [9]:
def score_and_rank(traces, score_fcn, key_candidates, partitions):
        dtype = [('key', int), ('score', 'float64')]
        ranks = []
        # for each key partition        
        for i in range(partitions): 
            partition_scores = np.array([], dtype=dtype)
            
            # for each key guess in the partition score the value and add to list
            for k in key_candidates:
                score_k = score_fcn(traces, k)
                key_score = np.array([(k, score_k)], dtype=dtype)
                partition_scores = np.append(partition_scores, key_score)
                
            # rank each key where partition_ranks[0] is the key that scored the highest
            partition_ranks = np.array([key_score[0] for key_score in np.sort(partition_scores, order='score')[::-1]])
            
            ranks.append(partition_ranks)
        return ranks

## Success Rate
In the analysis of a system, the Success Rate metric can be used alongside the Score and Rank metrics in order to help determine the security of a system. This metric is typically conducted by an evaluator not an attacker since the correct key for the cryptographic system must be known. The success rate of a given experiment, i, is defined as 1 if the correct key is ranked within the top o key guesses. The order, o, can be any value between 1 and K where K is the number of key guesses. When o is 1, the success rate is only 1 if the correct key was ranked first out of all possible key guesses. Similarly, if o is 2, the success rate will be 1 if the correct key is ranking within the top 2 ranks. 
\begin{equation}
    SR_{o}^{i}=
    \begin{cases}
        \text{1 if } k_{c} \in [guess_{1}, guess_{2}, ..., guess_{o}]\\
        \text{0 otherwise}
    \end{cases}
\end{equation}
The overall success rate is defined as the sum of the success rates of all experiments divided by the number of experiments. Lower success rates indicate a high degree of system security and vise versa. 
\begin{equation}
    SR_{o}= \frac{1}{p}\sum_{i=1}^{p}SR_{o}^{i}
\end{equation}

## Guessing Entropy
Guessing Entropy is another similar metric that can analyze the security of a system. The Guessing Entropy is defined as the sum of the natural log of the rank of the correct key for all experiments. 
\begin{equation}
    GE^{i}=log_{2}(rank_{k_{c}})
\end{equation}
\begin{equation}
    GE = \frac{1}{p}\sum_{i=1}^{p}GE^{i}
\end{equation}
The guessing entropy metric conveys the average workload left in the attack. As the guessing entropy decreases the certainty of the key guess increases, to the point where at a GE of 0, the key guess is certain.

In [10]:
def success_rate_guessing_entropy(correct_key, ranks, order, num_experiments):
    success_rate = 0
    guessing_entropy = 0
    
    # for each experiment
    for i in range(num_experiments):
        
        # check if correct key is within o ranks
        for j in range(order):
            if ranks[i][j] == correct_key:
                success_rate += 1
                break
        
        # guessing entropy is the log2 of the rank of the correct key
        guessing_entropy += math.log2(ranks[i].index(correct_key) + 1)
    
    success_rate = success_rate / num_experiments
    guessing_entropy = guessing_entropy / num_experiments
    
    return success_rate, guessing_entropy

## Pearson Correlation Coefficient
Source: Hardware Hacking Handbook: https://github.com/HardwareHackingHandbook/notebooks/blob/main/labs/HHH_10_Splitting_the_Difference_Differential_Power_Analysis.ipynb
Correlation can be used as a metric that compares two different trace sets. The first set of power traces are observed leakages relating to an intermediate value V. The second set are predicted power traces that were derived using some sort of leakage model g(.) relating to an intermediate algorithm output value V for the correct key guess.
$$
p_{k_{c}} = \frac{\sum_{i=1}^{n} (l_{i}- \frac{1}{n} \sum_{i=1}^{n} l_{i}) (g(f(x_{i},k_{c})) - \frac{1}{n} \sum_{i=1}^{n} g(f(x_{i},k_{c})))}{{\sqrt{\sum_{i=1}^{n}({l_i - \frac{1}{n}\sum_{i=1}^n}{l_i})^2} \sqrt{\sum_{i=1}^n{(g(f(x_{i},k_{c}))-\frac{1}{n} \sum_{i=1}^{n}{g(f(x_{i},k_{c}))})^2}}}}
$$
If the absolute value of the correlation between predicted traces modeled using the correct key, is greater than that of other key guesses, then the key will likely be able to be derived from a side channel attack. The pearson correlation can be used to create a correlation trace. The key can be derived by plotting the correlation trace for different key guesses. The correlation trace modeled using the correct key will have a spike in magnitude of the correlation. 

In [11]:
def pearson_correlation(predicted_leakage, observed_leakage, num_traces, num_samples):
    
    predicted_mean = np.mean(predicted_leakage, axis=0)
    observed_mean = np.mean(observed_leakage, axis=0)
    
    numerator = np.zeros(num_samples)
    denominator1 = np.zeros(num_samples)
    denominator2 = np.zeros(num_samples)
    
    for d in range(num_traces):
        l = observed_leakage[d] - observed_mean
        g = predicted_leakage[d] - predicted_mean
        
        numerator = numerator + g * l 
        denominator1 = denominator1 + np.square(l)
        denominator2 = denominator2 + np.square(g)
        
    correlation_trace = numerator / np.sqrt(denominator1 * denominator2)    
    
    return correlation_trace

## T-Test with TVLA
The goal of the t-test is to assess a device’s security, drawing conclusions regarding its relative venerability of lack thereof. A useful TVLA configuration for side channel analysis includes testing a device with a fixed key by sending deterministic and non-deterministic plaintext values. The resulting traces are separated into two different subsets, $L_{rand}$ and $L_{fixed}$ corresponding to if they were recorded from fixed or random plaintext values. Using statistical hypothesis testing where H0 corresponds to the device being secure and H1 indicating that the device has security flaws, the following hypothesis test can let us draw security conclusions. 
$$
\hat{\mu}_i = \frac{1}{n_i} \sum_{l \in L_i} l \ \ where \ \ i \in \{rand, fixed\}
$$

$$
H_0: \hat{\mu}_{rand} = \hat{\mu}_{fixed} \ \ H_1: \hat{\mu}_{rand} \neq \hat{\mu}_{fixed}
$$

The t-statistic value, t, can then be calculated via the following equation.

$$
\hat{\sigma}^2_{i} = \frac{1}{n_i -1}\sum_{l \in L_i}(l - \hat{\mu}_i)^2
$$

$$
t = \frac{\hat{\mu}_{rand} - \hat{\mu}_{fixed}}
{\sqrt{\frac{\hat{\sigma}^2_{rand}}{n_{rand}} - \frac{\hat{\sigma}^2_{fixed}}{n_{fixed}}}}
$$

$H_0$ will be rejected if $|t|$ is greater than some threshold $th$. This threshold is commonly set $th = 4.5$, a value that minimizes the possibility of Type I errors. Therefore, we can conclude that the DUT is leaking information if the t-statistic passes the given threshold.

In [13]:
def t_test_tvla(fixed, random, num_traces):
    
    # calculate t-statistic for each trace
    t_stats = []    
    for t in range(num_traces):
        t_statistic, p_value = stats.ttest_ind(fixed[t], random[t], axis=0, equal_var=False)
        t_stats.append(t_statistic)
    
    # high t-statistic and low p-values indicate that a given time sample leaks information
    return t_stats